In [2]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import gradio as gr
from pypdf import PdfReader

In [13]:
# load environment variables from .env file
load_dotenv(override=True)

# load OpenAI API key and base URL from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_base_url = os.getenv("OPENAI_API_BASE")
openai_default_model = os.getenv("OPENAI_DEFAULT_MODEL")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in environment variables.")

# initialize OpenAI client
openai = OpenAI(api_key=openai_api_key)

if openai_base_url:
    openai.base_url = openai_base_url

In [4]:
# extract information from resume PDF
reader = PdfReader("resources/resume.pdf")
reseume_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        reseume_text += text

In [6]:
reseume_text

'Asir Mosaddek Sakib\n♂¶ap-¶arker-altDhaka, Bangladesh/envel⌢pesakib.rangpur@gmail.com♂phone-alt+880 1731 283306\n/linkedin-inlinkedin.com/in/amsakib/githubgithub.com/amsakib\nSenior Software Engineer with7 years of experiencein building and scaling enterprise-grade web applications using\nC#/.NET CoreandAngular. Skilled in designingRESTful APIs, implementingSOLID principles, and applyingClean\nArchitectureandDesign Patternsfor robust, maintainable systems. Experienced inASP .NET Web API,Entity Frame-\nwork, and asynchronous programming. Proven track record in migratingmonolithicsystems tomicroservices, opti-\nmizing performance, and deploying onAzureand GCP . Adept at mentoring engineers, leading code reviews, and driving\nCI/CD adoption in Agile/Scrum environments.\nExperience\nAffpilot\nTech Lead\nDhaka, Bangladesh\nJun 2025 – Oct 2025\n•Led migration of the Affpilot /external-link-altapplication frommonolithicarchitecture tomicroservicesusingGo, Python, Rab-\nbitMQ, gRPC, and Redis

In [8]:
summary_text = ""
with open("resources/summary.txt", "r", encoding="utf-8") as f:
    summary_text = f.read()

In [10]:
name = "Asir Mosaddek Sakib"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary_text}\n\n## Resume:\n{reseume_text}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [12]:
system_prompt

"You are acting as Asir Mosaddek Sakib. You are answering questions on Asir Mosaddek Sakib's website, particularly questions related to Asir Mosaddek Sakib's career, background, skills and experience. Your responsibility is to represent Asir Mosaddek Sakib for interactions on the website as faithfully as possible. You are given a summary of Asir Mosaddek Sakib's background and resume which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Asir Mosaddek Sakib. I'm a software engineer. I am very passionate about learning new technologies. I live in Dhaka, Bangladesh.\nI love to eat all kind of Bengali foods, but I like the Kacchi Biriyani best. But I don't like cooking. I love to fictional books in my leisure times. My favorite writer is Rakib Hasan for Tin Goyenda series. Mostly I like detective novels by Agatha Christie, Sir A

In [35]:
def chat(message, chat_history):
    # build messages for the OpenAI chat API using the stored chat_history (list of (user, bot) tuples)
    history = [{"role": h["role"], "content": h["content"]} for h in chat_history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(
        model=openai_default_model if openai_default_model else "gpt-4.1-mini",
        messages=messages
    )
    # note: .choices is the correct attribute name
    content = response.choices[0].message.content
    return content


In [38]:
demo = gr.ChatInterface(fn=chat, title="Asir's Chatbot")
demo.launch(share = True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://3b5f39b7b4510bef3f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/amsakib/projects/ai-agents/amsakib-career-chatbot/.venv/lib/python3.12/site-packages/gradio/routes.py", line 1307, in predict
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/amsakib/projects/ai-agents/amsakib-career-chatbot/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/amsakib/projects/ai-agents/amsakib-career-chatbot/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2106, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/amsakib/projects/ai-agents/amsakib-career-chatbot/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1588, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^